# Chargement de données
## Auteur Linda MARTIN

La constrution des données à charger dans le model sont à catégoriser. Un modéle qui prend en entrée 558 colonnes et 424 en sortie, sera trop couteux en mémoire et en temps d'apprentissage.
La structure de travail combine le fichier de test et d'entraiment. La séparation des données d'appentissage et de test se fait après la struturation. Le modéle doit être valider avec des données différents de celle de l'apprentissage.


In [1]:
# import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [5]:
# load data
train_df = pd.read_csv('../data/train.csv', index_col= 'date_id')
test_df = pd.read_csv('../data/test.csv', index_col = 'date_id')

train_labels_df = pd.read_csv('../data/train_labels.csv', index_col = 'date_id')
target_pairs_df = pd.read_csv('../data/target_pairs.csv')



In [6]:
display(train_df.tail())
display(test_df.tail())
display(train_df.shape, test_df.shape)

,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,JPX_RSS3_Rubber_Futures_Open,...,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
date_id,,,,,,,,,,,,,,,,,,,,,
1912,2450.0,9523.5,1961.5,2676.5,15086.0,15440.0,15085.0,4461.5,4467.0,311.7,...,1.864661,0.598318,0.827529,0.495125,0.049224,0.072574,0.080968,0.046175,14.058107,0.041388
1913,2471.5,9519.5,1980.5,2710.5,15165.0,15509.0,15162.0,4495.0,4490.0,324.2,...,1.863539,0.594400,0.824390,0.490018,0.049409,0.072828,0.080671,0.046113,14.082236,0.041630
1914,2471.5,9533.5,1974.0,2693.0,15040.0,15477.0,15044.0,4544.5,4555.0,322.6,...,1.860067,0.595250,0.822392,0.489529,0.049095,0.073232,0.081083,0.045901,14.126606,0.041457
1915,2456.0,9500.5,1970.0,2697.5,15420.0,15752.0,15420.0,4670.0,4685.0,320.2,...,1.859624,0.597780,0.817224,0.488520,0.049205,0.073018,0.081170,0.045987,14.095322,0.041368
1916,2463.5,9610.0,1991.0,2701.5,15416.0,15802.0,15415.0,4730.5,4751.0,322.0,...,1.860472,0.597644,0.822219,0.491394,0.049321,0.073058,0.081233,0.046037,14.107311,0.041404


,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,JPX_RSS3_Rubber_Futures_Open,...,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP,is_scored
date_id,,,,,,,,,,,,,,,,,,,,,
1912,2450.0,9523.5,1961.5,2676.5,15086.0,15440.0,15085.0,4461.5,4467.0,311.7,...,0.598318,0.827529,0.495125,0.049224,0.072574,0.080968,0.046175,14.058107,0.041388,True
1913,2471.5,9519.5,1980.5,2710.5,15165.0,15509.0,15162.0,4495.0,4490.0,324.2,...,0.594400,0.824390,0.490018,0.049409,0.072828,0.080671,0.046113,14.082236,0.041630,True
1914,2471.5,9533.5,1974.0,2693.0,15040.0,15477.0,15044.0,4544.5,4555.0,322.6,...,0.595250,0.822392,0.489529,0.049095,0.073232,0.081083,0.045901,14.126606,0.041457,True
1915,2456.0,9500.5,1970.0,2697.5,15420.0,15752.0,15420.0,4670.0,4685.0,320.2,...,0.597780,0.817224,0.488520,0.049205,0.073018,0.081170,0.045987,14.095322,0.041368,True
1916,2463.5,9610.0,1991.0,2701.5,15416.0,15802.0,15415.0,4730.5,4751.0,322.0,...,0.597644,0.822219,0.491394,0.049321,0.073058,0.081233,0.046037,14.107311,0.041404,True


(1917, 557)

(90, 558)

Nous avons les 90 dernières dates du fichier d'entrainement qui correspond aux données de tests.

In [7]:
# on supprime les id_data du fichier test pour le jeu de donnée
cutoff_date = train_df.index[-90]
real_train_df = train_df[train_df.index < cutoff_date]
display(real_train_df.tail())

,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,JPX_RSS3_Rubber_Futures_Open,...,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
date_id,,,,,,,,,,,,,,,,,,,,,
1822,2571.5,9091.5,1975.0,2868.0,13620.5,13895.0,13624.0,4802.0,4798.0,363.3,...,1.772281,0.630820,0.806869,0.508989,0.051018,0.071398,0.079822,0.047927,13.745669,0.042869
1823,2578.5,9094.5,1958.5,2865.5,NaN,NaN,NaN,NaN,NaN,NaN,...,1.756813,0.637624,0.804638,0.513056,0.051247,0.071740,0.080362,0.048176,13.807214,0.043007
1824,2560.0,9154.0,1966.0,2862.5,13701.5,13960.0,13701.0,4790.5,4796.0,368.6,...,1.752560,0.635810,0.804070,0.511236,0.051273,0.071971,0.080197,0.048221,13.890139,0.043275
1825,2601.5,9167.5,1935.0,2863.0,13583.5,13873.0,13587.0,4740.0,4757.0,376.6,...,1.753901,0.636714,0.806343,0.513410,0.051739,0.072166,0.080590,0.048587,13.803503,0.043508
1826,2636.5,9230.5,1970.5,2874.5,13635.0,13918.0,13621.0,4690.0,4683.0,383.7,...,1.761285,0.632962,0.807244,0.510955,0.051597,0.071946,0.080207,0.048433,13.677595,0.043445


## Restructuration par caractéristiques
On restructure de la manière à avoir les colonnes par catégories de produit :
['date', 'id', 'close', 'open', 'high', 'low', 'volume', 'sprice', 'interest']

In [8]:

warnings.filterwarnings("ignore")
# Future block-specific warnings
warnings.filterwarnings(
    'ignore',
    category=FutureWarning,
    message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated."
)
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in ")

def get_preprocess_data(df, cond):
    # Fonction pour obtenir les données prétraitées en fonction de la condition
    if cond.Column.size > 0:
        return df[cond.Column.values[0]]
    else:
        return None
    
def get_train_info(df):
    """ Construction d'une data des entêtes de colonnes.
    Args:
        df (pd.DataFrame): Input dataframe d'entrainement.
    Returns:
        pd.DataFrame: Détail des informations de chaque colonne.
    """
    df_names = df.columns
    # Fonction pour nettoyer et split les noms
    def clean_and_split(name):
        name = name.replace("open_interest", "open interest")
        name = name.replace("settlement_price", "settlement price")
        name = name.replace("US_Stock", "US Stock")
        name = name.replace("adj_close", "Close")
        name = name.replace("adj_", "adjusted ")
        name = name.replace("-", "_")
        return name.split("_")

    # Création du DataFrame d'infos
    df_info = pd.DataFrame(
        {
        'Column': df_names,
        'Split': [clean_and_split(name) for name in df_names]
    })

    df_info['Category'] = df_info['Split'].apply(lambda x: x[0])    
    df_info['Ticker'] = df_info['Split'].apply(
    lambda x: "_".join(x[1:-1]) if len(x) > 2 else x[-1] if len(x) == 2 else ""
    )
    df_info['Type'] = df_info['Split'].apply(lambda x: x[-1])

    # Nettoyage final
    df_info['Ticker'] = df_info.apply(
        lambda row: row['Type'] if row['Ticker'] == "" else row['Ticker'], axis=1
    )
    df_info['Column_Id'] = df_info.index + 1

    # Sélection des colonnes finales
    df_info = df_info[['Column_Id', 'Column', 'Category', 'Ticker', 'Type']]    
    return df_info

def preprocess(df):
    """ Pré-process pour regrouper les données par catégorie.
     Args:
        df (pd.DataFrame): Input dataframe d'entrainement.
        Returns: pd.DataFrame par catégorie."""
    
    df_processed = df.copy()
    df_info = get_train_info(df_processed)

    df_processed.reset_index(inplace=True)
    # On renomme la date_id en date
    df_processed = df_processed.rename({'date_id': 'date'}, axis='columns')
    # Initialisation du DataFrame résultat
    result = pd.DataFrame(columns=['date', 'id', 'close', 'open', 'high', 'low', 'volume', 'sprice', 'interest'])
    
    for  Category  in df_info.groupby('Category').groups.keys():
         for label in df_info[(df_info.Category==Category)].groupby('Ticker').groups.keys():
            temp_df = pd.DataFrame()
            temp_df['date'] = df_processed['date']
            temp_df['id'] = f'{Category}_{label}'

            if Category in ['FX','LME']:
                temp_df['close'] = df_processed[df_info[(df_info.Category==Category) & (df_info.Ticker==label)].Column.values[0]]
                temp_df['open'] = None
                temp_df['high'] = None
                temp_df['low'] = None
                temp_df['volume'] = None
                temp_df['sprice'] = None
                temp_df['interest'] = None
            else:
                temp_df['close'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Close', 'adjusted close']))])
                temp_df['open'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Open','adjusted open']))])
                temp_df['high'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['High','adjusted high']))])
                temp_df['low'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Low','adjusted low']))])
                temp_df['volume'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['Volume', 'adjusted volume']))])
                temp_df['sprice'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['settlement price','adjusted settlement price']))])
                temp_df['interest'] = get_preprocess_data(df_processed,df_info[(df_info.Category==Category) & (df_info.Ticker==label) & (df_info.Type.isin(['open interest','adjusted open interest']))])
            result = pd.concat([result, temp_df], ignore_index=True)
      
    # Réinitialiser l'index
    result = result.reset_index(drop=True)
    
    # Trier par date et id 
    result = result.sort_values(['date', 'id']).reset_index(drop=True)
    
    return result
        


In [ ]:
df_preprocess = preprocess(real_train_df)
display(df_preprocess.head())

,date,id,close,open,high,low,volume,sprice,interest
0,0,FX_AUDCAD,0.979601,NaN,NaN,NaN,NaN,NaN,NaN
1,0,FX_AUDCHF,0.761027,NaN,NaN,NaN,NaN,NaN,NaN
2,0,FX_AUDJPY,87.933498,NaN,NaN,NaN,NaN,NaN,NaN
3,0,FX_AUDNZD,1.103011,NaN,NaN,NaN,NaN,NaN,NaN
4,0,FX_AUDUSD,0.783393,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
261256,1826,US Stock_X,35.709500,36.6072,36.8466,35.6496,4944039.0,NaN,NaN
261257,1826,US Stock_XLB,87.343900,86.6970,87.3538,86.4581,5775459.0,NaN,NaN
261258,1826,US Stock_XLE,92.518100,91.5457,92.6024,91.4365,12615301.0,NaN,NaN
261259,1826,US Stock_XOM,109.326200,109.1985,109.6896,108.5700,10366585.0,NaN,NaN


A cette structure on ajoute des fonctions comme le déplacement de données pour le traitement de lag et des moyennes dans différentes zones de temps sur chaque colonne.  

In [ ]:
from typing import List

class FeatureEngineer:
    """Class to handle feature engineering"""
    def __init__(self):
        pass

    def add_lag_features(self,
        df: pd.DataFrame, 
        lags: List[int], 
        date_col: str = 'date'
        ) -> pd.DataFrame:
        """
        Add lag features for specified columns and lags.
        """
        df = df.sort_values(date_col)
        cols = set(df.columns)
        cols.remove('id')
        cols.remove('date')
        for col in cols:
            for lag in lags:
                df[f'{col}_lag{lag}'] = df.groupby('id')[col].shift(lag)
        return df
    
    def add_rolling_features(self,
        df: pd.DataFrame,
        windows: List[int],
        date_col: str = 'date') -> pd.DataFrame:
        """ 
        Add rolling mean and std features for specified columns and windows.
        """
        df = df.sort_values(date_col)
        cols = set(df.columns)
        cols.remove('id')
        cols.remove('date')
        for col in cols:
            for window in windows:
                df[f'{col}_rollmean{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).mean())
                df[f'{col}_rollstd{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).std())
                df[f'{col}_rollmin{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).min())
                df[f'{col}_rollmax{window}'] = df.groupby('id')[col].transform(lambda x: x.rolling(window, min_periods=1).max())
       
        return df
    
    def prepare_features(self, train_df):
        """Engineer features for training and testing data"""
        try:
            # Combine train and test for feature engineering
            train_cols = set(train_df.columns)
            
            combined_data = self.add_lag_features(train_df, lags=[1, 2, 3, 5, 7])
            
            # Add rolling features
            combined_data = self.add_rolling_features(combined_data, windows=[5, 10, 15])
           
            # Handle missing values
            combined_data = combined_data.fillna(method='ffill').fillna(method='bfill').fillna(0)

            return combined_data
        except Exception as e:
            print(f"Feature preparation failed: {e}")
            raise

In [ ]:
feature_engineer = FeatureEngineer()
train_features = feature_engineer.prepare_features(df_preprocess)
display(train_features.head())

Train features shape: (261261, 548)
Test features shape: (0, 548)
✓ Feature preparation completed!


array([[0, 'FX_AUDCAD', 0.979601, ..., 0.27414529906602114, 147.0081,
        147.0081],
       [0, 'FX_AUDJPY', 87.933498, ..., 0.27414529906602114, 147.0081,
        147.0081],
       [0, 'FX_AUDNZD', 1.103011, ..., 0.27414529906602114, 147.0081,
        147.0081],
       ...,
       [1826, 'US Stock_VCSH', 76.7651, ..., 0.11807629269616528,
        76.2543, 76.6865],
       [1826, 'FX_ZARGBP', 0.043445, ..., 0.11807629269616528, 76.2543,
        76.6865],
       [1826, 'US Stock_YINN', 25.8436, ..., 1.340138913836175, 24.8661,
        29.574]], shape=(261261, 548), dtype=object)

# Create train/test groups using DataLoader

In [14]:
from sklearn.preprocessing import LabelEncoder
# Convertir en DataFrame
df = pd.DataFrame(train_features)
df.columns = ["date", "instrument"] + [f"f{i}" for i in range(df.shape[1]-2)]
# Encoder l’instrument (Embedding-friendly)
le = LabelEncoder()
df["instrument_id"] = le.fit_transform(df["instrument"])
# Trier par instrument + date (OBLIGATOIRE)
df = df.sort_values(["instrument_id", "date"])

# Séparer les features numériques
feature_cols = [c for c in df.columns if c.startswith("f")]
X_num = df[feature_cols].astype("float32").values
inst = df["instrument_id"].values.astype("int64")




In [15]:
from torch.utils.data import Dataset

class LSTMDataset(Dataset):
    def __init__(self, X, inst, seq_len):
        self.X = X
        self.inst = inst
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx:idx+self.seq_len]),
            torch.tensor(self.inst[idx+self.seq_len])
        )
    
dataset = LSTMDataset(X_num, inst, seq_len=30)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
# prepare_targets(self):
print("Preparing targets...")
target_cols = [col for col in train_labels_df.columns if col not in ['timestamp', 'id']]
display(target_cols)
target_values = train_labels_df[target_cols].values

Preparing targets...


['target_0',
 'target_1',
 'target_2',
 'target_3',
 'target_4',
 'target_5',
 'target_6',
 'target_7',
 'target_8',
 'target_9',
 'target_10',
 'target_11',
 'target_12',
 'target_13',
 'target_14',
 'target_15',
 'target_16',
 'target_17',
 'target_18',
 'target_19',
 'target_20',
 'target_21',
 'target_22',
 'target_23',
 'target_24',
 'target_25',
 'target_26',
 'target_27',
 'target_28',
 'target_29',
 'target_30',
 'target_31',
 'target_32',
 'target_33',
 'target_34',
 'target_35',
 'target_36',
 'target_37',
 'target_38',
 'target_39',
 'target_40',
 'target_41',
 'target_42',
 'target_43',
 'target_44',
 'target_45',
 'target_46',
 'target_47',
 'target_48',
 'target_49',
 'target_50',
 'target_51',
 'target_52',
 'target_53',
 'target_54',
 'target_55',
 'target_56',
 'target_57',
 'target_58',
 'target_59',
 'target_60',
 'target_61',
 'target_62',
 'target_63',
 'target_64',
 'target_65',
 'target_66',
 'target_67',
 'target_68',
 'target_69',
 'target_70',
 'target_71',
 '

In [21]:
def train_models_for_target(target_name, y):
    """Train models for a specific target.
    Args:
        target_name (str): Name of the target variable.
        y (np.ndarray): Target values.
    Returns:
        dict: Trained models for the target.
    """
    models = {}
    try:
        # Example: Train a simple model (placeholder)
        print(f"  Training models for target: {target_name}")
        
        # Remove NaN targets
        valid_indices = ~np.isnan(y)
        y_valid = target_values[valid_indices]
        display(y_valid)
        display(valid_indices)
        
        # Placeholder for model training logic
        return models
    except Exception as e:
        print(f"Failed to train models for {target_name}: {e}")
        return {}
    
# train_all_models
models = {}
num_targets = min(5, len(target_cols))
selected_targets = target_cols[:num_targets]
print(f"Training models for {num_targets} targets...")
for target_idx, target_name in enumerate(selected_targets):
    print(f"\nTarget {target_idx + 1}/{num_targets}: {target_name}")
    
    # Get target values
    y = target_values[:, target_idx]
    
    # Train models for this target
    target_models = train_models_for_target(target_name, y)
    
    if target_models:
        models[target_name] = target_models
        print(f"  ✓ Trained {len(target_models)} models for {target_name}")
    else:
        print(f"  ✗ No models trained for {target_name}")
        

Training models for 5 targets...

Target 1/5: target_0
  Training models for target: target_0


array([[ 0.0059485 , -0.00285132, -0.0046752 , ...,  0.03823403,
                nan,  0.02730989],
       [ 0.00578281, -0.02411763, -0.00705199, ...,  0.02502068,
         0.00354846,  0.02094043],
       [ 0.00104825,  0.02383639, -0.00893406, ...,  0.00483537,
        -0.00907498,  0.00170587],
       ...,
       [-0.01259616,  0.00930876,  0.00794689, ...,         nan,
         0.03100571,         nan],
       [-0.00073629, -0.00129211, -0.00513709, ..., -0.00805691,
        -0.00206856, -0.14105322],
       [-0.00229414,  0.01289763,  0.00997804, ..., -0.01330409,
        -0.00552677, -0.12768791]], shape=(1787, 424))

array([ True,  True,  True, ...,  True, False, False], shape=(1917,))

  ✗ No models trained for target_0

Target 2/5: target_1
  Training models for target: target_1


array([[ 0.0059485 , -0.00285132, -0.0046752 , ...,  0.03823403,
                nan,  0.02730989],
       [ 0.00578281, -0.02411763, -0.00705199, ...,  0.02502068,
         0.00354846,  0.02094043],
       [ 0.00104825,  0.02383639, -0.00893406, ...,  0.00483537,
        -0.00907498,  0.00170587],
       ...,
       [-0.01259616,  0.00930876,  0.00794689, ...,         nan,
         0.03100571,         nan],
       [-0.00073629, -0.00129211, -0.00513709, ..., -0.00805691,
        -0.00206856, -0.14105322],
       [-0.00229414,  0.01289763,  0.00997804, ..., -0.01330409,
        -0.00552677, -0.12768791]], shape=(1744, 424))

...

array([ True,  True,  True, ...,  True, False, False], shape=(1917,))

  ✗ No models trained for target_3

Target 5/5: target_4
  Training models for target: target_4


array([[ 0.00578281, -0.02411763, -0.00705199, ...,  0.02502068,
         0.00354846,  0.02094043],
       [-0.00327176,  0.00523447,  0.00685561, ...,         nan,
         0.00951448,         nan],
       [ 0.00731558, -0.00770781, -0.01662585, ...,  0.01237079,
         0.01882961, -0.01263144],
       ...,
       [-0.01259616,  0.00930876,  0.00794689, ...,         nan,
         0.03100571,         nan],
       [-0.00073629, -0.00129211, -0.00513709, ..., -0.00805691,
        -0.00206856, -0.14105322],
       [-0.00229414,  0.01289763,  0.00997804, ..., -0.01330409,
        -0.00552677, -0.12768791]], shape=(1631, 424))

array([False,  True, False, ...,  True, False, False], shape=(1917,))

  ✗ No models trained for target_4
